In [1]:
# define rooms and items

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}


key_a = {
    "name": "key for door a",
    "type": "key",
    "target": 'door a',
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": 'door b',
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": 'door c',
}

key_d = {
    "name": "key for door d",
    "type": "key",
    "target": 'door d',
}

piano = {
    "name": "piano",
    "type": "furniture",
}

couch = {
    "name": "couch",
    "type": "furniture",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_01 = {
    "name": "bedroom 1",
    "type": "room",
}

bedroom_02 = {
    "name": "bedroom 2",
    "type": "room",
}

living_room = {
    "name": "living room",
    "type": "room",
}

outside = {
  "name": "outside"
}

all_rooms = [game_room, bedroom_01, bedroom_02, living_room, outside]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "bedroom 1" : [queen_bed, door_b, door_c],
    "bedroom 2" : [double_bed, dresser, door_b],
    "double bed" : [key_c],
    "queen bed" : [key_b],
    "dresser" : [key_d],
    'living room' : [dining_table, door_d],
    "outside" : [door_d],
    "door a" : [game_room, bedroom_01],
    "door b" : [bedroom_01, bedroom_02],
    "door c" : [bedroom_01, living_room],
    'door d' : [living_room, outside]
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

from datetime import datetime

import time

from rich import print

In [15]:
def player_input(prompt, possible_inputs):
    intended_action = ''

    while intended_action not in possible_inputs:
      try:
        intended_action = input(prompt).strip().lower()

        if intended_action not in possible_inputs:
          raise ValueError("Selected action not available")

      except ValueError as error:
        print("Error: ", error)


    return intended_action

def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    print("You wake up on a couch and find yourself in a strange house\nwith no windows which you have never been to before.\nYou don't remember why you are here and what had happened before.\nYou feel some unknown danger is approaching and you [bold underline red]must\nget out of the house, NOW[/bold underline red]!")
    time.sleep(2)
    print("\n")
    print("Don't hurry me, I need to wake up too! Now, we can get started...")
    time.sleep(2)
    print("\n")
    print("┏(-_-)┛┗(-_- )┓┗(-_-)┛┏(-_-)┓")
    time.sleep(2)
    play_room(game_state["current_room"], True)

def play_room(room, first_time):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    game_state["current_room"] = room

    if(game_state["current_room"] == game_state["target_room"]):
        print("\n")
        print("Congrats! You escaped the room!")
    else:
        print("\n")
        if first_time:
          print("You are in the " + room["name"])
        inputslist = ["explore", "date", "examine"]
        print("\n")
        print("You can explore the room with 'explore',")
        print("You can examine an item with 'examine', or")
        print("You can check today's date with 'date'.\n")

        intended_action = player_input("What would you like to do? ", inputslist)

        if intended_action == "explore":
            print("\n")
            explore_room(room)
            play_room(room, False)

        elif intended_action == "date":
            now = datetime.now()
            print("\n")
            print(f"Oookay, why do you even need to know this now, just check your calendar - today is the {now:%d.%m.%Y}\n\nBUT more importantly!!: time is running, so you better choose a different option now!")
            play_room(room, False)

        elif intended_action == "examine":
            print("\n")
            items = [i["name"] for i in object_relations[room["name"]]]
            print("Here you find '" + "', '".join(items) + "'")
            print("\n")
            examine_item(input("Which object would you like to examine? ").strip()).lower()
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is the " + room["name"] + ". You find '" + "', '".join(items) + "'")

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None

    for item in object_relations[current_room["name"]]:
        if item["name"] == item_name:
            print("\n")
            output = "You are now examining " + item_name + ". "
            if item["type"] == "door":
                have_key = False
                for key in game_state["keys_collected"]:
                    if key["target"] == item['name']:
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "[bold underline red]It is locked but you don't have the key.[/bold underline red]"
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]][0]
                    game_state["keys_collected"].append(item_found)
                    output += "You find the " + item_found["name"] + "."
                else:
                    output += "Sadly, this" + " " + item["name"] + " " + "is rather boring. Stay curios!"
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no' ").strip().lower() == 'yes'):
        play_room(next_room, True)
    else:
        play_room(current_room, False)
    linebreak()



In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house
with no windows which you have never been to before.
You don't remember why you are here and what had happened before.
You feel some unknown danger is approaching and you must
get out of the house, NOW!

Don't hurry me, I need to wake up too! Now, we can get started...

┏(-_-)┛┗(-_- )┓┗(-_-)┛┏(-_-)┓

You are in the game room

You can explore the room with 'explore',

...

Here you find 'couch', 'piano', 'door a'

You are now examining door a. It is locked but you don't have the key.

You can explore the room with 'explore',

You can examine an item with 'examine', or

You can check today's date with 'date'.